**Task 09: Data linking**

In [177]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4/"

In [178]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g3 = Graph()
g1.parse(github_storage+"resources/data03.rdf", format="xml")
g2.parse(github_storage+"resources/data04.rdf", format="xml")

<Graph identifier=Ne6008b00911842bd90abd4060e01084b (<class 'rdflib.graph.Graph'>)>

Busca individuos en los dos grafos y enlázalos mediante la propiedad OWL:sameAs, inserta estas coincidencias en g3. Consideramos dos individuos iguales si tienen el mismo apodo y nombre de familia. Ten en cuenta que las URI no tienen por qué ser iguales para un mismo individuo en los dos grafos.

**Task 1: Find same individuals in graphs 1 and 2 and link them using OWL:sameAs and insert the properties in g3**

Conditions to fulfill in order to be considered equal:
*   Same Given
*   Same Family name (Family)




In [179]:
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import OWL
base = Namespace("http://data.five.org#") #base g3
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
q1 = prepareQuery('''
  SELECT 
    ?Subject ?Given ?Family
  WHERE { 
    ?Subject vcard:Given ?Given.
    ?Subject vcard:Family ?Family
  }
  ''',
  initNs = { "vcard": VCARD}
)
#Initialization of the arrays that will be used to store given, uri and family values of graphs 1 and 2.
given1 = []
given2 = []
uri1 = []
uri2 = []
family1 = []
family2 = [] 
#Retrieval of the values obtained through the query
for r in g1.query(q1):
  uri1.append(r.get(0))          #retrieval of the URI value
  given1.append(str(r.get(1)))   #retrieval of the Given value
  family1.append(str(r.get(2)))  #retrieval of the Family value
for r in g2.query(q1):
  uri2.append(r.get(0))          #retrieval of the URI value
  given2.append(str(r.get(1)))   #retrieval of the Given value
  family2.append(str(r.get(2)))  #retrieval of the Family value

#Size comparison of the lengths of two arrays, one from each class to check which one is bigger.
#Accordingly the biggest size is stored in the "size" variable and the smallest in the "size1" variable.
#given1 is used but any other array would be okay because in the query we make sure that the size is the same in all arrays.
size=len(given1)
size1=len(given2)  
if(min(len(given1), len(given2))!=len(given1)):
  size=len(given2)
  size1=len(given1)  
  
#Traversal of all arrays and if conditions are fulfilled triples are added to the corresponding graph
for i in range(size):
  index = i
  for j in range(size1):
    index1 = j
    #In order to avoid traversing out of bounds we check if the indexes are okay if not, the value is changed into the correct one.
    if size != len(given1):
      index = j
      index1 = i
    if size == len(given1) and given1[index] == given2[index1] and family1[index] == family2[index1]:
      uri = URIRef(base+given1[index]+family1[index])
      g3.add((uri,VCARD.Given,Literal(given1[index])))
      g3.add((uri,VCARD.Family,Literal(family1[index])))
      g1.add((uri1[index],OWL.sameAs,uri2[index1]))
      g2.add((uri2[index1],OWL.sameAs,uri1[index]))
  
print("Graph 3")
for s,p,o in g3:
  print(s,p,o)

print("Graph 2")
for s,p,o in g2:
 if(p==OWL.sameAs):
    print(s,p,o)

print("Graph 1")
for s,p,o in g1: 
 if(p==OWL.sameAs):
    print(s,p,o)



[rdflib.term.URIRef('http://data.three.org#SaraJones'), rdflib.term.URIRef('http://data.three.org#JohnSmith'), rdflib.term.URIRef('http://data.three.org#JohnDoe')]
Graph 3
http://data.five.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://data.five.org#JohnDoe http://www.w3.org/2001/vcard-rdf/3.0#Family Doe
http://data.five.org#SaraJones http://www.w3.org/2001/vcard-rdf/3.0#Family Jones
http://data.five.org#SaraJones http://www.w3.org/2001/vcard-rdf/3.0#Given Sara
Graph 2
http://data.four.org#0005 http://www.w3.org/2002/07/owl#sameAs http://data.three.org#JohnDoe
http://data.four.org#0001 http://www.w3.org/2002/07/owl#sameAs http://data.three.org#SaraJones
Graph 1
http://data.three.org#SaraJones http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0001
http://data.three.org#JohnDoe http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0005
